# 門禁系統實作

我們將在這個課程中，使用 Azure 臉部辨識的功能，實作門禁系統。  
這個教學分成兩部分：
1. 第一部分：說明如何使用 Azure 臉部辨識的功能，完成辨識是兩張人臉是否一樣的功能。門禁系統最重要的元件，就是辨識目前的人臉，是否為系統中所預先儲存好的人臉。也就是說，要能辨識出兩張照片中的人臉，是否為屬同一個人，即使這兩張照片是在不同的角度，光影環境下，依然是可以運作。
2. 第二部分：第一部分所完成的功能，只是辨識兩張照片的人臉是否一致。但並沒有「人」的概念。這是說，系統只知道有「臉」，但沒有法子把「臉」關聯到「人」。在第二部分中，我們要把這部分補上。這樣系統才有法子識別出目前這位使用者的人臉，是否為系統中登錄合法的「人」。

參考資料:
- [微軟FACE API](https://westus.dev.cognitive.microsoft.com/docs/services/563879b61984550e40cbbe8d/operations/563879b61984550f3039523a)
- [課程講義--臉部辨識實作](http://elearning.nkust.edu.tw/learn/path/SCORM_fetchResource.php)

In [1]:
# 匯入相關套件
import requests
import json
from io import BytesIO
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

請依照前一講的內容，將金鑰，以及 url 填入。

In [2]:
subscription_key = ''
face_api_detect_url = 'https://<xxxxxxx>.cognitiveservices.azure.com/face/v1.0/detect'
face_api_verify_url = 'https://<xxxxxxx>.cognitiveservices.azure.com/face/v1.0/verify'

## Part 1: 辨識兩張照片中的人臉是否是同一張臉

請找兩張照片，人臉儘可能清晰，且屬於同一人。

這裏以``劉若英``的照片為例。

底下有 5 張照片，都是劉若英的照片。  
我們使用 img1-img4 當作模型的訓練資料，以 img5 做為測試資料

In [3]:
img1 = 'https://www.rumtoast.com/wp-content/uploads/2019/02/%E8%9E%A2%E5%B9%95%E5%BF%AB%E7%85%A7-2019-02-19-%E4%B8%8B%E5%8D%885.10.29_meitu_1.jpg'
img2 = 'https://lifestyle.etnet.com.hk/column/images/stories/72/2020/05/20200504lolo01.png'

img3 = 'https://image.pttnews.cc/2018/04/11/cb4bcfe702/5d5f72667741987975c9005514ceaab5.jpg'
img4 = 'https://imgs.gvm.com.tw/upload/gallery/20180601/44545_05.jpg'
img5 = 'https://cdn2.ettoday.net/images/3586/d3586654.jpg'

將兩張照片送到 Azure 雲端，分別取得這兩張照片的 Faceid。

In [30]:
#第一張照片
headers = {'Ocp-Apim-Subscription-Key': subscription_key}
#使用 recognition_02 的辨識模型，這個模型是 2019 年開發出來，比 recognition_01 有更精準的辨識度
params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'recognitionModel':'recognition_02'
}
response = requests.post(face_api_detect_url, params=params,
                         headers=headers, json={"url": img1})
print(json.dumps(response.json()))
#取得第一張照片的 faceid
face = response.json()[0]
faceid1 = face['faceId']
print(faceid1)

[{"faceId": "288ce2ab-3c4e-4f6b-bed9-77be1180d695", "faceRectangle": {"top": 140, "left": 478, "width": 175, "height": 175}}]
288ce2ab-3c4e-4f6b-bed9-77be1180d695


In [31]:
#第二張照片
headers = {'Ocp-Apim-Subscription-Key': subscription_key}

params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'recognitionModel':'recognition_02'
}
response = requests.post(face_api_detect_url, params=params,
                         headers=headers, json={"url": img2})
print(json.dumps(response.json()))
#取得第二張照片的 faceid
face = response.json()[0]
faceid2 = face['faceId']
print(faceid2)

[{"faceId": "6c44cb1b-c839-469b-884e-e2605bb4f154", "faceRectangle": {"top": 77, "left": 286, "width": 108, "height": 108}}]
6c44cb1b-c839-469b-884e-e2605bb4f154


### 使用 verify 的功能，判定是否為一樣的人臉

上面我們使用了 Azure 的 detect (偵測) 的功能，由照片中抓出人臉，並取得一個唯一的人臉 id。  
接著我們使用 Azure 的 verify (辨識) 功能，判定兩張照片中的人臉，是否為同一張人臉。

如同 detect 功能，verify 功能也對應到一個雲端的網址，其網址如下

https://<端點>/face/v1.0/**verify**

大家可以看到網址的最後面是 verify 字串，代表是要呼叫 verify 的功能。  
如果把 verify 改為 detect，那就是要呼叫 detect 的功能。

In [32]:
# 設定要傳給雲端的參數
headers = {'Ocp-Apim-Subscription-Key': subscription_key}
params = {
    'faceId1': faceid1,
    'faceId2': faceid2
}
#記得呼叫 verify 的網址！
body=json.dumps(params)

response = requests.post(face_api_verify_url, headers=headers, data=body)
print(json.dumps(response.json()))

{"isIdentical": true, "confidence": 0.87974}


在上例中，這兩張照片中的人臉被視為同一張臉。  
微軟 AI 模型的信心有 88%。

如果使用 recognition_01 的模型的話，其信心只有 50% 出頭而已。 

## Par2: 建構門禁系統

在第 2 部分中，我們要開始建構門禁系統。  
門禁系統通常要辨識一組使用者。例如家庭的門禁系統通常會包含一群家人的人臉。  

為了完成這個功能，我們要完成底下的步驟：  
1. 建立人臉群組 (personGroupId)  
2. 建立一位使用者 (person)  
3. 將使用者加入群組中  
4. 將人臉加入對應的使用者中 (這步驟中，一位使用者最多可以加入 248 張臉)  
5. 訓練模型，使 Azure 可以將所有的人臉和對應的使用者關聯起來  
6. 辨識  

### 建立人臉群組

[建立人臉群組的參考資料](https://westus.dev.cognitive.microsoft.com/docs/services/563879b61984550e40cbbe8d/operations/563879b61984550f30395244)

In [7]:
#使用者的群組名稱
#給一個使用者群組的 id，長度最長為 64 個字元，只能包含小寫的英文字母，以及 - 或 _
personGroupId = 'home3'

In [8]:
face_api_personGroup_url = 'https://testiemfaceapi.cognitiveservices.azure.com/face/v1.0/persongroups/' + personGroupId

headers = {'Ocp-Apim-Subscription-Key': subscription_key}

# recognitionModel 代表辨識臉部所用的 AI 模型，微軟目前推薦 recognition_02。
# 這是 2019 年開發出來的模型，其精確度較 recognition_01 這個舊的模型來得高。
body = {
    'name':'friends',
    'recognitionModel':'recognition_02'
}
body1=json.dumps(body)

response = requests.put(face_api_personGroup_url, headers=headers, data=body1)
print(response)
#print(json.dumps(response.json()))

<Response [200]>


上面的程式碼若 response 為 200，代表成功地建立了群組

### 建立一位使用者，並將這位使用者加入群組中

In [15]:
face_api_person_url = 'https://<xxxxxxx>.cognitiveservices.azure.com/face/v1.0/persongroups/' + personGroupId +'/persons'
headers = {'Ocp-Apim-Subscription-Key': subscription_key}
body = {
    'name':'girl1'
}
body1=json.dumps(body)
response = requests.post(face_api_person_url, headers=headers, data=body1)
d1 = response.json()


In [17]:
#執行成功後，得到一個 personId。
#這個 PersonId 是新加入使用者的唯一編號。

personid = d1['personId']
print(personid)

99f0c79a-e865-45f8-97ca-568e95aa4ca1


### 替使用者加入對應的臉部照片

我們將 img1-img4 加入 girl1 這位使用者的照片庫中。  

[加入臉部資料的文件](https://westus.dev.cognitive.microsoft.com/docs/services/563879b61984550e40cbbe8d/operations/563879b61984550f3039523b)

In [21]:
face_api_addface_url = 'https://<xxxxxxx>.cognitiveservices.azure.com/face/v1.0/persongroups/' + personGroupId +'/persons/'+personid+'/persistedFaces'
headers = {'Ocp-Apim-Subscription-Key': subscription_key}
#第一張臉
response = requests.post(face_api_addface_url, headers=headers, json={'url':img1})
face = json.dumps(response.json())
print(face)
#第二張臉
response = requests.post(face_api_addface_url, headers=headers, json={'url':img2})
face = json.dumps(response.json())
print(face)
#第三張臉
response = requests.post(face_api_addface_url, headers=headers, json={'url':img3})
face = json.dumps(response.json())
print(face)
#第四張臉
response = requests.post(face_api_addface_url, headers=headers, json={'url':img4})
face = json.dumps(response.json())
print(face)

{"persistedFaceId": "c3e2b2fe-6239-4d63-8d8c-24ca1eae68c1"}
{"persistedFaceId": "9b7dc194-1be6-47eb-ab81-acc5f6167557"}
{"persistedFaceId": "77522957-a0eb-483d-8110-098be7c4b43a"}
{"persistedFaceId": "af276813-2e6c-49f6-8167-2c3e91ea7148"}


上述的程式碼是可以用 for 迴圈完成！

### 訓練模型

臉都加入後，就可以開始訓練模型了

In [22]:
face_api_train_url='https://<xxxxxxx>.cognitiveservices.azure.com/face/v1.0/persongroups/' + personGroupId +'/train'
headers = {'Ocp-Apim-Subscription-Key': subscription_key}
response = requests.post(face_api_train_url, headers=headers)
print(response)

<Response [202]>


上面的程式碼要是回傳 202，表示 Azure 已經接受了你的要求。

我們可以呼叫底下的程式碼，看看模型訓練好了沒？

In [24]:
face_api_training_url='https://<xxxxxxx>.cognitiveservices.azure.com/face/v1.0/persongroups/' + personGroupId +'/training'
headers = {'Ocp-Apim-Subscription-Key': subscription_key}
response = requests.get(face_api_training_url, headers=headers)
print(json.dumps(response.json()))

{"status": "succeeded", "createdDateTime": "2020-05-11T12:02:09.7551379Z", "lastActionDateTime": "2020-05-11T12:02:10.1272205Z", "message": null, "lastSuccessfulTrainingId": "df843044-07a3-481a-979e-56e76e323317", "lastSuccessfulTrainingDateTime": "2020-05-11T12:02:10.1272205Z"}


如果上述的狀態是 successed，那表示完成訓練了。  
如果是 running，表示還在訓練中。   
如果是 notstarting，表示還沒開始。  
如果是 failed，表示失敗。  

In [28]:
#第五張照片
headers = {'Ocp-Apim-Subscription-Key': subscription_key}

params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'recognitionModel':'recognition_02'
}
response = requests.post(face_api_detect_url, params=params,
                         headers=headers, json={"url": img5})
print(json.dumps(response.json()))
#取得第一張照片的 faceid
face = response.json()[0]
faceid5 = face['faceId']
print(faceid5)

[{"faceId": "409e7f8b-4cef-484e-b6c9-00d24e7bd98d", "faceRectangle": {"top": 68, "left": 308, "width": 109, "height": 109}}]
409e7f8b-4cef-484e-b6c9-00d24e7bd98d


將第5張照片傳給雲端，並使用 Persongroup 來辨識，能不能辦識成功。

In [29]:
# 設定要傳給雲端的參數
headers = {'Ocp-Apim-Subscription-Key': subscription_key}
body = {
    'faceId': faceid5,
    'personGroupId': personGroupId,
    'personId': personid
}
#記得呼叫 verify 的網址！
body1=json.dumps(body)

response = requests.post(face_api_verify_url, headers=headers, data=body1)
print(json.dumps(response.json()))

{"isIdentical": true, "confidence": 0.94516}


在上面的例子中，微軟的雲端模型有約 95% 的信心，這是群組中的人臉。

用另一張「非群組」中的人臉，看看微軟的判斷是如何？

In [33]:
#第 6 張照片，使用于文文的照片
img6 = 'https://images.chinatimes.com/newsphoto/2018-02-22/900/BBC200_P_04_02.jpg'
headers = {'Ocp-Apim-Subscription-Key': subscription_key}

params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'recognitionModel':'recognition_02'
}
response = requests.post(face_api_detect_url, params=params,
                         headers=headers, json={"url": img6})
print(json.dumps(response.json()))
#取得第一張照片的 faceid
face = response.json()[0]
faceid6 = face['faceId']
print(faceid6)

[{"faceId": "047219a7-c923-4c45-846e-ead6df42f3bc", "faceRectangle": {"top": 210, "left": 429, "width": 271, "height": 271}}]
047219a7-c923-4c45-846e-ead6df42f3bc


In [34]:
# 設定要傳給雲端的參數
headers = {'Ocp-Apim-Subscription-Key': subscription_key}
body = {
    'faceId': faceid6,
    'personGroupId': personGroupId,
    'personId': personid
}
#記得呼叫 verify 的網址！
body1=json.dumps(body)

response = requests.post(face_api_verify_url, headers=headers, data=body1)
print(json.dumps(response.json()))

{"isIdentical": false, "confidence": 0.09591}


由上述的結果可以看出，Azure 判斷這張照片中的人臉是不屬於群組中的人臉。  
信心只有約 9%。